In [1]:
from nltk.tokenize import word_tokenize
import pandas as pd

In [7]:
df_main = pd.read_excel(open('/home/karthikshivaram/Desktop/Test Date.xlsx','rb'), engine='openpyxl',sheet_name='Source data 1')

In [8]:
df_main

,Accountid,Transaction ID,Narrative
0,Ab8654,xxx123,"Accrual Jun 20, Receipt, code 6785"
1,Ab8654,xxx786,"Credited Apr21, draft, xxxxx"
2,CB8754,yyy9876,credit draft yyyy Paid
3,CB8734,yyy1065,accountable new account receipt
4,CB8754,yyy2456,"open credit , new receipt"
5,MS9845,vvv6543,credit discounted
6,MS9845,vvv6547,"accrual Aug 20, discounted"


In [10]:
df_terms = pd.read_excel(open('/home/karthikshivaram/Desktop/Test Date.xlsx','rb'), engine='openpyxl',sheet_name='Source data 2')


In [11]:
df_terms

,Account ID,Tag word list
0,Ab8654,Credit
1,Ab8654,Accrual
2,Ab8654,Discounted
3,Ab8654,Credited
4,CB8754,Paid
5,CB8754,Open credit
6,CB8754,New receipt
7,CB8754,discounted
8,CB8734,Accrual
9,CB8734,discounted


In [14]:
terms_dict = {}
for grp, df_grp in df_terms.groupby("Account ID"):
    print(df_grp.columns)
    terms_dict[grp] = df_grp["Tag word list"].tolist()

Index(['Account ID', 'Tag word list'], dtype='object')
Index(['Account ID', 'Tag word list'], dtype='object')
Index(['Account ID', 'Tag word list'], dtype='object')
Index(['Account ID', 'Tag word list'], dtype='object')


In [15]:
terms_dict

{'Ab8654': ['Credit ', 'Accrual', 'Discounted', 'Credited'],
 'CB8734': ['Accrual', 'discounted'],
 'CB8754': ['Paid', 'Open credit', 'New receipt', 'discounted'],
 'MS9845': ['discounted', 'credit', 'accrual']}

In [58]:
from nltk.tokenize import word_tokenize
import pandas as pd

def find_terms(filepath='/home/karthikshivaram/Desktop/Test Date.xlsx'):
    """
    """
    df_main = pd.read_excel(open(filepath,'rb'), engine='openpyxl',sheet_name='Source data 1')
    df_terms = pd.read_excel(open(filepath,'rb'), engine='openpyxl',sheet_name='Source data 2')
    
    terms_dict = {}
    for grp, df_grp in df_terms.groupby("Account ID"):
        terms_dict[grp] = df_grp["Tag word list"].apply(lambda x: x.lower()).tolist()
    
    df_main["tokens"] = df_main["Narrative"].apply(lambda x: x.lower()).apply(lambda x: word_tokenize(x))
    
    tagwords = []
    for index,row in df_main.iterrows():
        tagwords.append(list(set(terms_dict[row.Accountid]).intersection(row.tokens)))
    
    df_main["Tagword Chk"] = tagwords
    df_main.drop(columns=['tokens'],inplace=True)
    return df_main

In [59]:
find_terms(filepath='/home/karthikshivaram/Desktop/Test Date.xlsx')

Index(['Accountid', 'Transaction ID', 'Narrative', 'tokens'], dtype='object')


,Accountid,Transaction ID,Narrative,Tagword Chk
0,Ab8654,xxx123,"Accrual Jun 20, Receipt, code 6785",[accrual]
1,Ab8654,xxx786,"Credited Apr21, draft, xxxxx",[credited]
2,CB8754,yyy9876,credit draft yyyy Paid,[paid]
3,CB8734,yyy1065,accountable new account receipt,[]
4,CB8754,yyy2456,"open credit , new receipt",[]
5,MS9845,vvv6543,credit discounted,"[credit, discounted]"
6,MS9845,vvv6547,"accrual Aug 20, discounted","[discounted, accrual]"
